# Groupings of Year Built for Buildings

**Author:** Han Zhang  
**Date:** November 13, 2025    
**Data Source:** Chicago Energy Benchmarking Dataset (2014-2023) - City of Chicago Open Data Portal  

# Data Loading

In [ ]:
# load in local helper functions
import altair as alt
import pandas as pd

from utils.data_utils import clean_property_type, concurrent_buildings, load_data
from utils.plot_utils import (
    plot_delta_property_chart,
    plot_energy_persistence_by_year,
    plot_energy_persistence_rows,
    plot_metric_by_property,
)

alt.data_transformers.disable_max_rows()

# Load the data loader function
energy_df = load_data()

# Energy subset that includes only concurrent buildings appear across 2016-2023
energy_df = concurrent_buildings(energy_df, 2016, 2023)
energy_df = clean_property_type(energy_df)

print(
    f"Loaded dataset with {energy_df.shape[0]:,} rows and {energy_df.shape[1]} columns."
)
energy_df.head()

Loaded dataset with 18,904 rows and 30 columns.


,Data Year,ID,Property Name,Address,ZIP Code,Community Area,Primary Property Type,Gross Floor Area - Buildings (sq ft),Year Built,# of Buildings,...,Total GHG Emissions (Metric Tons CO2e),GHG Intensity (kg CO2e/sq ft),Latitude,Longitude,Location,Reporting Status,Chicago Energy Rating,Exempt From Chicago Energy Rating,Water Use (kGal),Row_ID
24486,2016,116336,lasalle private residences,1212 N LaSalle,60610,near north side,multifamily housing,367627.0,1986.0,1.0,...,2575.3,7.0,41.904201,-87.633825,point (-87.63382507 41.90420084),nan,NaN,nan,NaN,nan
24496,2016,101745,161 north clark,161 North Clark,60601,loop,office,1200836.0,1992.0,1.0,...,15498.4,12.9,41.884905,-87.630518,point (-87.6305179 41.88490511),nan,NaN,nan,NaN,nan
24495,2016,101448,1401 w roosevelt - 2017 resubmit,1401 W. Roosevelt,60608,near west side,residential,69385.0,2006.0,1.0,...,242.4,3.5,41.849153,-87.670896,point (-87.67089596 41.84915346),nan,NaN,nan,NaN,nan
24494,2016,159892,promontory corporation,5530-5532 S Shore Drive,60637,hyde park,multifamily housing,180351.0,1949.0,1.0,...,1317.3,7.3,41.794687,-87.580465,point (-87.58046479 41.794687),nan,NaN,nan,NaN,nan
24493,2016,103602,190 south lasalle,190 South LaSalle,60603,loop,office,882560.0,1985.0,1.0,...,11519.9,13.1,41.879756,-87.632687,point (-87.63268685 41.8797561),nan,NaN,nan,NaN,nan


# Year Grouping

In [2]:
decade_start = (energy_df["Year Built"] // 10) * 10
energy_df["Decade Built"] = (
    decade_start.astype(str) + "-" + (decade_start + 9).astype(str)
)
energy_df.loc[energy_df["Year Built"].isna(), "Decade Built"] = None

building_counts = energy_df["Decade Built"].value_counts(dropna=False)
print(building_counts)

Decade Built
2000.0-2009.0    2747
1920.0-1929.0    2635
None             1924
1960.0-1969.0    1904
1970.0-1979.0    1621
1980.0-1989.0    1407
1990.0-1999.0    1336
2010.0-2019.0    1148
1910.0-1919.0    1083
1950.0-1959.0     920
1900.0-1909.0     670
1930.0-1939.0     657
1890.0-1899.0     403
1940.0-1949.0     263
1880.0-1889.0     135
1870.0-1879.0      39
1860.0-1869.0       7
2020.0-2029.0       4
1690.0-1699.0       1
Name: count, dtype: int64


In [ ]:
import numpy as np

def assign_effective_year_built(df: pd.DataFrame) -> pd.DataFrame:
    """Assigns the 'Effective Year Built' for each building ID.

    If one unique non-NaN year exists, it is assigned; if multiple years exist, assigns 'Multiple Years Built'; otherwise assigns np.nan.

    Args:
        df (pd.DataFrame): DataFrame with columns 'ID' and 'Year Built'.

    Returns:
        pd.DataFrame: Original DataFrame with new 'Effective Year Built' column.
    """

    def get_years(series: pd.Series) -> np.ndarray:
        unique_years = series.dropna().unique()
        if len(unique_years) == 1:
            # Building has one unique non-NaN value (regardless of number of NaNs)
            return np.repeat(unique_years[0], len(series))
        elif len(unique_years) > 1:
            # Building has multiple non-NaN values
            return np.repeat("Multiple Years Built", len(series))
        else:
            # Building has only NaNs
            return np.repeat(np.nan, len(series))

    df["Effective Year Built"] = df.groupby("ID")["Year Built"].transform(get_years)
    return df
    

energy_df = assign_effective_year_built(energy_df)

In [4]:
percentages = energy_df["Effective Year Built"].value_counts(normalize=True) * 100
print(percentages)

Effective Year Built
Multiple Years Built    18.958950
1928.0                   1.946678
1920.0                   1.819721
1929.0                   1.735083
1925.0                   1.692763
                          ...    
1873.0                   0.042319
1865.0                   0.042319
1880.0                   0.042319
1891.0                   0.042319
1888.0                   0.042319
Name: proportion, Length: 135, dtype: float64


In [5]:
is_valid = (energy_df["Effective Year Built"].notna()) & (
    energy_df["Effective Year Built"] != "Multiple Years Built"
)

energy_df = energy_df[is_valid].copy()

In [6]:
print(
    f"Loaded dataset with {energy_df.shape[0]:,} rows and {energy_df.shape[1]} columns."
)
energy_df.head()

Loaded dataset with 15,320 rows and 32 columns.


,Data Year,ID,Property Name,Address,ZIP Code,Community Area,Primary Property Type,Gross Floor Area - Buildings (sq ft),Year Built,# of Buildings,...,Latitude,Longitude,Location,Reporting Status,Chicago Energy Rating,Exempt From Chicago Energy Rating,Water Use (kGal),Row_ID,Decade Built,Effective Year Built
24486,2016,116336,lasalle private residences,1212 N LaSalle,60610,near north side,multifamily housing,367627.0,1986.0,1.0,...,41.904201,-87.633825,point (-87.63382507 41.90420084),nan,NaN,nan,NaN,nan,1980.0-1989.0,1986.0
24496,2016,101745,161 north clark,161 North Clark,60601,loop,office,1200836.0,1992.0,1.0,...,41.884905,-87.630518,point (-87.6305179 41.88490511),nan,NaN,nan,NaN,nan,1990.0-1999.0,1992.0
24495,2016,101448,1401 w roosevelt - 2017 resubmit,1401 W. Roosevelt,60608,near west side,residential,69385.0,2006.0,1.0,...,41.849153,-87.670896,point (-87.67089596 41.84915346),nan,NaN,nan,NaN,nan,2000.0-2009.0,2006.0
24494,2016,159892,promontory corporation,5530-5532 S Shore Drive,60637,hyde park,multifamily housing,180351.0,1949.0,1.0,...,41.794687,-87.580465,point (-87.58046479 41.794687),nan,NaN,nan,NaN,nan,1940.0-1949.0,1949.0
24493,2016,103602,190 south lasalle,190 South LaSalle,60603,loop,office,882560.0,1985.0,1.0,...,41.879756,-87.632687,point (-87.63268685 41.8797561),nan,NaN,nan,NaN,nan,1980.0-1989.0,1985.0


In [7]:
bins = [0, 1920, 1960, 1990, 2010, float("inf")]
labels = ["Before 1920", "1920-1960", "1960-1990", "1990-2010", "After 2010"]

energy_df["Decade Built"] = pd.cut(
    energy_df["Year Built"], bins=bins, labels=labels, right=False, include_lowest=True
)

percentages = energy_df["Decade Built"].value_counts(normalize=True) * 100
print(percentages)

Decade Built
1960-1990      29.837316
1920-1960      26.446876
1990-2010      24.337748
Before 1920    12.251656
After 2010      7.126404
Name: proportion, dtype: float64


# Visualizations

In [8]:
chart = plot_metric_by_property(
    df=energy_df,
    metric_col="Site EUI (kBtu/sq ft)",
    property_col="Decade Built",
    agg_func=pd.Series.mean,
    year_col="Data Year",
    marker_year=2019,
    width=700,
    height=500,
)

chart

/project/src/utils/plot_utils.py:641: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby([year_col, property_col], as_index=False)


alt.LayerChart(...)

In [9]:
chart = plot_metric_by_property(
    df=energy_df,
    metric_col="Site EUI (kBtu/sq ft)",
    property_col="Decade Built",
    agg_func=pd.Series.median,
    year_col="Data Year",
    marker_year=2019,
    width=700,
    height=500,
)

chart

/project/src/utils/plot_utils.py:641: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby([year_col, property_col], as_index=False)


alt.LayerChart(...)

In [10]:
chart = plot_delta_property_chart(
    df=energy_df,
    metric_col="Site EUI (kBtu/sq ft)",
    property_col="Decade Built",
    id_col="ID",
    year_col="Data Year",
    top_types=sorted(energy_df["Decade Built"].dropna().unique()),
    marker_year=2019,
    width=800,
    height=500,
)

chart

alt.LayerChart(...)

In [11]:
cols = ["ID", "Data Year", "Decade Built", "Site EUI (kBtu/sq ft)"]
site_df = energy_df[cols].dropna().copy()

site_df["Data Year"] = site_df["Data Year"].astype(int)
site_df["ID"] = site_df["ID"].astype(str)
site_df["Decade Built"] = site_df["Decade Built"].astype(str)
site_df["Site EUI (kBtu/sq ft)"] = pd.to_numeric(
    site_df["Site EUI (kBtu/sq ft)"], errors="coerce"
)
site_df = site_df.dropna(subset=["Site EUI (kBtu/sq ft)"])

df_delta = (
    site_df.sort_values(["ID", "Data Year"])
    .groupby("ID", group_keys=False)
    .apply(lambda g: g.assign(Delta=g["Site EUI (kBtu/sq ft)"].diff()))
    .dropna(subset=["Delta"])
    .reset_index(drop=True)
)

/tmp/ipykernel_115/4107217643.py:15: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: g.assign(Delta=g["Site EUI (kBtu/sq ft)"].diff()))


In [12]:
# Align delta_t and delta_t+1 for each building
df_lagged = (
    df_delta.sort_values(["Decade Built", "ID", "Data Year"])
    .groupby(["Decade Built", "ID"])
    .apply(
        lambda g: g.assign(
            Delta_next=g["Delta"].shift(-1)  # change from N+1→N+2
        )
    )
    .dropna(subset=["Delta", "Delta_next"])
    .reset_index(drop=True)
)
df_lagged.head()

/tmp/ipykernel_115/1981874856.py:5: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(


,ID,Data Year,Decade Built,Site EUI (kBtu/sq ft),Delta,Delta_next
0,100001,2017,1920-1960,182.6,-7.8,6.2
1,100001,2018,1920-1960,188.8,6.2,-10.5
2,100001,2019,1920-1960,178.3,-10.5,-3.3
3,100001,2020,1920-1960,175.0,-3.3,-4.0
4,100001,2021,1920-1960,171.0,-4.0,-7.0


In [13]:
corrs = (
    df_lagged.groupby("Decade Built")[["Delta", "Delta_next"]]
    .corr()
    .iloc[0::2, -1]
    .reset_index()
    .rename(columns={"Delta_next": "Persistence (Δₜ → Δₜ₊₁)"})
)
corrs

,Decade Built,level_1,Persistence (Δₜ → Δₜ₊₁)
0,1920-1960,Delta,-0.082343
1,1960-1990,Delta,-0.152239
2,1990-2010,Delta,-0.405704
3,After 2010,Delta,-0.333456
4,Before 1920,Delta,-0.339995


In [ ]:
charts = plot_energy_persistence_rows(
    df_lagged=df_lagged,
    property_col="Decade Built",
    id_col="ID",
    year_col="Data Year",
    delta_col="Delta",
    delta_next_col="Delta_next",
)

for chart in charts:
    chart.show()

KeyError: 0

In [ ]:
from utils.plot_utils import plot_energy_persistence_by_year

chart = plot_energy_persistence_by_year(
    df_lagged=df_lagged,
    property_col="Decade Built",
    id_col="ID",
    year_col="Data Year",
    delta_col="Delta",
    delta_next_col="Delta_next",
)

chart